# Teste completo: Baseline CNN vs Híbrido CNN+VQC (Colab + GPU)

Este notebook roda no **Google Colab** com GPU para replicar a comparação do artigo (Pereira & Saraiva, 2021):
- **Baseline:** 3 runs × 100 épocas, cenários No Balance e ROS (altere N_RUNS_* na célula 5 para 10 se quiser replicar o artigo).
- **Híbrido:** mesmo número de runs.

**Antes de rodar:** ative o runtime com GPU (Runtime → Change runtime type → GPU).

### O que voce pode alterar no notebook

1. **Celula 2 (clone):** Se voce fez upload de um ZIP do projeto em vez de clonar, troque por: `!unzip -q -o /content/electricity-theft-hybrid-qml.zip -d /content` e `%cd /content/electricity-theft-hybrid-qml`

2. **Celula 4 (Kaggle):** Opcao A = usar Colab Secrets (KAGGLE_USERNAME, KAGGLE_KEY). Opcao B = descomente as 3 ultimas linhas e faca upload do kaggle.json

3. **Celula 5 (Carregar dados):**
- **N_RUNS_BASELINE** e **N_RUNS_HYBRID** = 3 (iguais para comparacao justa). Use 10 para replicar o artigo.
- **EPOCHS** = 100 (padrao artigo). Reduza (ex: 5) para teste rapido.
- **SEED** = 42 (reprodutibilidade).
- **Dataset ja carregado:** Se voce subiu a pasta do dataset no Colab, defina o caminho antes de get_dataset_path, ex: `import os; os.environ["SGCC_DATASET_PATH"] = "/content/drive/MyDrive/sgcc"` (ou o caminho da sua pasta).

## 1. Configurar ambiente e clonar o repositório

### Tempo estimado e como nao desligar / nao desconectar

**Tempo aproximado:** Com **3 runs × 2 cenários = 6 runs** por modelo (100 épocas):
- **Baseline (GPU):** ~2–5 min por run → **~15–30 min** no total.
- **Híbrido (CNN+VQC):** O VQC em si não é “lento”; o que pesa é **simular o circuito quântico na CPU** (PennyLane). Por run pode levar **~15–45 min** → com 3 runs, **~1h30–2h30** no total. Com 10 runs seria bem mais longo.

Para teste ainda mais rápido: use menos épocas (ex: 5).

**Para o PC nao desligar (rodando localmente no Linux):**
- **Desativar suspensao:** Configuracoes → Energia → “Quando inativo” = Nunca (ou apenas desligar tela).
- **Via terminal:** `systemctl mask sleep.target suspend.target hibernate.target hybrid-sleep.target` (reverter depois com `unmask`).
- **Impedir que a tela bloqueie so durante o treino:** use `caffeinate` (macOS) ou no Linux algo como `xset s off; xset -dpms` antes de rodar (reverte depois com `xset s on; xset +dpms`), ou instale “Caffeine” / “Inhibit” no seu ambiente de desktop.

**No Google Colab:**
- O Colab pode desconectar apos ~90 min de inatividade. Mantenha a aba aberta e, se quiser, use um “keep-alive” no navegador (extensoes que simulam atividade) ou um script que rode em loop no console (ex: clicar no codigo a cada alguns minutos). Nao ha garantia; para runs muito longos, prefira rodar por partes (menos runs por vez) ou salvar checkpoints.

In [ ]:
# Verificar GPU (TensorFlow usa automaticamente no Colab)
import tensorflow as tf
print("GPU disponível:", tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

In [ ]:
# Clonar repositório (substitua pela URL do seu fork se necessário)
!git clone https://github.com/alanveloso/electricity-theft-hybrid-qml.git 2>/dev/null || true
%cd electricity-theft-hybrid-qml

In [ ]:
# Instalar dependências (inclui pennylane-lightning para possível aceleração)
!pip install -q numpy pandas scikit-learn tensorflow kagglehub pennylane pennylane-lightning matplotlib

## 2. Configurar Kaggle (dataset SGCC)

Opção A: Colab Secrets - em Key adicione KAGGLE_USERNAME e KAGGLE_KEY (ou KAGGLE_API_TOKEN).
Opção B: Faça upload do kaggle.json (Account, Create New Token no Kaggle) na célula abaixo.

In [ ]:
import os

# Opção A: usar secrets do Colab (Key = KAGGLE_USERNAME, KAGGLE_KEY ou KAGGLE_API_TOKEN)
try:
    from google.colab import userdata
    if os.environ.get("KAGGLE_KEY") is None:
        os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")
    if os.environ.get("KAGGLE_USERNAME") is None:
        os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
    print("Credenciais Kaggle carregadas dos Secrets.")
except Exception as e:
    print("Secrets não configurados:", e)
    print("Opção B: faça upload do kaggle.json e descomente as linhas abaixo.")
    # from google.colab import files
    # uploaded = files.upload()  # escolha kaggle.json
    # !mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

## 3. Carregar dados e parâmetros do teste completo

In [ ]:
import sys
sys.path.insert(0, ".")

from src.data.sgcc_loader import load_sgcc_from_path, train_test_split
from train_baseline_pereira import get_dataset_path

# Parâmetros do teste completo (como no artigo)
N_RUNS_BASELINE = 3    # 10 no artigo; 3 para teste mais rápido
N_RUNS_HYBRID = 3      # mesmo número para comparação justa
EPOCHS = 100
SEED = 42
VERBOSE = 0  # 0 = quiet (recomendado no Colab)

path = get_dataset_path(None)
print("Carregando dataset SGCC...")
X, y = load_sgcc_from_path(path, seed=SEED, preprocessing="pereira")
print(f"X.shape = {X.shape}, y.shape = {y.shape}")
print(f"Normal: {(y==0).sum()}, Fraude: {(y==1).sum()}")

## 4. Rodar Baseline (CNN) — 3 runs × 100 épocas

In [ ]:
from train_baseline_pereira import run_scenario as run_baseline_scenario
import numpy as np

results_baseline = []
for scenario in ["no_balance", "ros"]:
    aucs, accs, times = [], [], []
    for run in range(N_RUNS_BASELINE):
        run_seed = SEED + run
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=True, seed=run_seed
        )
        res, _, _ = run_baseline_scenario(
            scenario, X_train, y_train, X_test, y_test,
            epochs=EPOCHS, verbose=VERBOSE, learning_rate=0.01, use_class_weight=False
        )
        aucs.append(res["auc"])
        accs.append(res["accuracy"])
        times.append(res["train_time_seconds"])
        print(f"  Baseline {scenario} run {run+1}/{N_RUNS_BASELINE} - AUC: {res['auc']:.4f}, Acc: {res['accuracy']*100:.2f}%, Tempo: {res['train_time_seconds']:.0f}s")
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs, ddof=1) if len(aucs) > 1 else 0
    mean_acc = np.mean(accs) * 100
    total_time = sum(times)
    mean_time = np.mean(times)
    results_baseline.append({"scenario": scenario, "mean_auc": mean_auc, "std_auc": std_auc, "mean_acc": mean_acc, "total_time_s": total_time, "mean_time_s": mean_time})
    print(f"  -> Media AUC: {mean_auc:.4f} +- {std_auc:.4f}, Media Acc: {mean_acc:.2f}% | Tempo total: {total_time:.0f}s ({mean_time:.0f}s/run)")
print("Baseline concluido.")

## 5. Rodar Híbrido (CNN+VQC) — 3 runs × 100 épocas

*(Mesmo número de runs do baseline para comparação justa. O circuito quântico é simulado em CPU; o treino da CNN usa GPU.)*

**Execução em computador IBM:** defina no ambiente `QML_DEVICE=qiskit.ibmq`, `QML_IBMQ_BACKEND=...` e `IBMQX_TOKEN=...`. O código configura e valida o device antes do treino (se falhar, o erro aparece logo).

In [ ]:
from train_hybrid_pereira import run_hybrid_scenario

results_hybrid = []
for scenario in ["no_balance", "ros"]:
    aucs, accs, times = [], [], []
    for run in range(N_RUNS_HYBRID):
        run_seed = SEED + run
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=True, seed=run_seed
        )
        res = run_hybrid_scenario(
            scenario, X_train, y_train, X_test, y_test,
            epochs=EPOCHS, verbose=VERBOSE, learning_rate=0.01
        )
        aucs.append(res["auc"])
        accs.append(res["accuracy"])
        times.append(res["train_time_seconds"])
        print(f"  Hibrido {scenario} run {run+1}/{N_RUNS_HYBRID} - AUC: {res['auc']:.4f}, Acc: {res['accuracy']*100:.2f}%, Tempo: {res['train_time_seconds']:.0f}s")
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs, ddof=1) if len(aucs) > 1 else 0
    mean_acc = np.mean(accs) * 100
    total_time = sum(times)
    mean_time = np.mean(times)
    results_hybrid.append({"scenario": scenario, "mean_auc": mean_auc, "std_auc": std_auc, "mean_acc": mean_acc, "total_time_s": total_time, "mean_time_s": mean_time})
    print(f"  -> Media AUC: {mean_auc:.4f} +- {std_auc:.4f}, Media Acc: {mean_acc:.2f}% | Tempo total: {total_time:.0f}s ({mean_time:.0f}s/run)")
print("Hibrido concluido.")

## 6. Tabela de comparação e referência do artigo

In [ ]:
import pandas as pd

rows = []
for r in results_baseline:
    rows.append({
        "Modelo": "Baseline (CNN)",
        "Cenario": r["scenario"],
        "Media AUC": f"{r['mean_auc']:.4f} +- {r['std_auc']:.4f}",
        "Media Acuracia (%)": f"{r['mean_acc']:.2f}",
        "Runs": N_RUNS_BASELINE,
        "Tempo total (s)": round(r["total_time_s"], 0),
        "Tempo medio/run (s)": round(r["mean_time_s"], 0)
    })
for r in results_hybrid:
    rows.append({
        "Modelo": "Hibrido (CNN+VQC)",
        "Cenario": r["scenario"],
        "Media AUC": f"{r['mean_auc']:.4f} +- {r['std_auc']:.4f}",
        "Media Acuracia (%)": f"{r['mean_acc']:.2f}",
        "Runs": N_RUNS_HYBRID,
        "Tempo total (s)": round(r["total_time_s"], 0),
        "Tempo medio/run (s)": round(r["mean_time_s"], 0)
    })

df = pd.DataFrame(rows)
display(df)

total_baseline = sum(r["total_time_s"] for r in results_baseline)
total_hybrid = sum(r["total_time_s"] for r in results_hybrid)
print(f"Tempo total Baseline: {total_baseline/60:.1f} min | Tempo total Hibrido: {total_hybrid/60:.1f} min")
print("Referencia (Pereira & Saraiva 2021, 10 runs, 100 epocas):")
print("  No Balance: AUC 0,5162 +- 0,0045   Acuracia 91,59%")
print("  ROS:        AUC 0,6714 +- 0,0062   Acuracia 67,78%")

In [ ]:
# Salvar resultados em CSV (download opcional)
df.to_csv("colab_full_test_results.csv", index=False)
print("Resultados salvos em colab_full_test_results.csv")